In [1]:
# imports
import keras
import tensorflow as tf
from keras.layers import Dense
from keras import Sequential
from tensorflow.keras.models import Model
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import ReLU
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
import numpy as np

In [1]:
def make_vgg16(dropout=None, train_layers=False, output_layer=False):
    """
    Creates a VGG16 CNN with custom top layers.

    Paramters: 
        dropout: float - dropout rate for all dropout layers. Must be between 0 and 1, default None (no dropout layers)
        train_layers: Boolean - whether or not to train the layers within the VGG16 network. Default False
        output_layer: Boolean - inclusion of final output layer with a single node and sigmoid activation. Default False

    return: 
        Keras model - VGG16 Keras model with custom top layers 
    """
    # VGG16 network
    base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', pooling='avg')

    # optional training of VGG16 layers
    if not train_layers:
        for layer in base_model.layers:
            layer.trainable = False
    else:
        for layer in base_model.layers:
            layer.trainable = True

    # flatten final VGG16 layer
    x = base_model.output
    x = Flatten()(x)
    
    # generate fully-connected layers with optional dropout layers
    x = Dense(4096, activation='relu')(x)
    if dropout is not None:
        x = Dropout(dropout)(x)
    x = Dense(4096, activation='relu')(x)
    if dropout is not None:
        x = Dropout(dropout)(x)
    x = Dense(1000, activation='relu')(x)
    if dropout is not None:
        x = Dropout(dropout)(x)
    x = Dense(500, activation='relu')(x)
    if dropout is not None:
        x = Dropout(dropout)(x)
    x = Dense(100, activation='relu')(x)
    if dropout is not None:
        x = Dropout(dropout)(x)

    
    x = Dense(10, activation='relu')(x)

    # final layer
    x = Dense(4, activation='relu')(x)

    # optional output layer if creating solo model
    if output_layer:
        x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=x)

    return model

In [5]:
# Simple NN
def make_nn(input_nodes, output_layer=False):
    """
    Creates a basic neural network model with 1 hidden layer.

    Paramters: 
        input_nodes: int - number of input nodes/dimensions/features 
        output_layer: Boolean - inclusion of final output layer with 2 nodes and softmax activation. Default False

    return: 
        Keras model - neural network Keras model
    """
    model = Sequential()
    model.add(Dense(input_nodes, input_dim=input_nodes, activation='relu'))

    model.add(Dense(input_nodes, activation='relu'))

    if output_layer:
        model.add(Dense(2, activation='softmax'))

    return model

In [6]:
def kt_model_build(hp):
    """
    Creates Keras-Tuner model for parameter searching/tuning. This function has the largest set of search parameters.

    Paramters: 
        None

    return: 
        Keras model - combined VGG16 and neural network model with selected search parameters 
    """
    # build basic NN first
    nn_model = Sequential()

    # activation functions to select from
    activation_list = ['relu', 'linear', 'elu', 'gelu', 'selu']

    # input nodes have to be same as total features, i.e., 4
    nn_input_nodes = 4

    # select NN input layer activation function
    input_activation = hp.Choice('input_activation', activation_list)
    
    # input NN layer
    nn_model.add(Dense(nn_input_nodes, input_dim=nn_input_nodes, activation=input_activation))

    # generate hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):

        layer_activation = hp.Choice(f'activation_{i}', activation_list)

        nn_model.add(Dense(units=hp.Int(f'nodes_{i}', min_value=3, max_value=36, step=1),
                        activation=layer_activation
        ))
    
    # final NN layer
    nn_model.add(Dense(4, activation='relu'))



    # build VGG16/CNN
    vgg_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', pooling='avg')

    # don't train VGG16 layers
    for layer in vgg_model.layers:
        layer.trainable = False

    x = vgg_model.output
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)

    # first set of layers with nodes between 1000-5000
    for i in range(hp.Int('cnn_set1_num_layers', 1, 3)):
        layer_activation = hp.Choice(f'cnn_set1_activation_{i}', activation_list)
        layer_nodes = hp.Int(f'cnn_set1_nodes_{i}', min_value=1000, max_value=5000, step=100)

        x = Dense(layer_nodes, activation=layer_activation)(x)
    
    # second set of layers with nodes between 10-500
    for i in range(hp.Int('cnn_set2_num_layers', 1, 3)):
        layer_activation = hp.Choice(f'cnn_set2_activation_{i}', activation_list)
        layer_nodes = hp.Int(f'cnn_set2_nodes_{i}', min_value=10, max_value=500, step=10)

        x = Dense(layer_nodes, activation=layer_activation)(x)


    # final CNN layer
    x = Dense(4, activation='relu')(x)

    cnn_model = Model(inputs=vgg_model.input, outputs=x)

    # combine models
    combined_model = tf.keras.layers.concatenate([nn_model.output, cnn_model.output], axis=1)
    combined_model = Dense(4, activation='relu')(combined_model)

    # final combined output layer
    combined_model = Dense(1, activation='sigmoid')(combined_model)

    final_model = Model(inputs=[nn_model.input, cnn_model.input], outputs=combined_model)

    # compile model
    # sample values for learning rate and epsilon
    lr = hp.Float("learning_rate", min_value=1e-5, max_value=1e-1, sampling="log")
    ep = hp.Float('epsilon', min_value=1e-9, max_value=1e-5, sampling='log')

    # using Adam optimizer
    opt = Adam(learning_rate=lr, epsilon=ep)

    # compile model
    final_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return final_model


In [7]:
def trimmed_kt_model_build(hp):
    """
    Creates Keras-Tuner model for parameter searching/tuning. This function has a trimmed set of search parameters.

    Paramters: 
        None

    return: 
        Keras model - combined VGG16 and neural network model with selected search parameters 
    """

    # build basic NN first - same general architecture as kt_model_build function
    nn_model = Sequential()

    # activation functions to select from
    activation_list = ['relu', 'gelu', 'selu']

    # input nodes have to be same # as total features, i.e., 4
    nn_input_nodes = 4

    input_activation = hp.Choice('input_activation', activation_list)
    nn_model.add(Dense(nn_input_nodes, input_dim=nn_input_nodes, activation=input_activation))

    # generate hidden layers
    for i in range(hp.Int('num_layers', 3, 5)):

        layer_activation = hp.Choice(f'activation_{i}', activation_list)

        nn_model.add(Dense(units=hp.Int(f'nodes_{i}', min_value=3, max_value=36, step=3),
                        activation=layer_activation
        ))
    
    final_nodes = 4
    nn_model.add(Dense(final_nodes, activation='relu'))


    # build CNN
    vgg_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', pooling='avg')

    for layer in vgg_model.layers:
        layer.trainable = False

    x = vgg_model.output
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)

    # sample dropout rate for dropout layers, values between 0.15-0.35
    droprate = hp.Float('droprate', min_value=0.15, max_value=0.35, step=0.05)
    x = Dropout(droprate)(x)

    # set of layers with nodes between 1000-4000, dropout layers after each 
    for i in range(hp.Int('cnn_set1_num_layers', 3, 6)):
        layer_activation = hp.Choice(f'cnn_set1_activation_{i}', activation_list)
        layer_nodes = hp.Int(f'cnn_set1_nodes_{i}', min_value=1000, max_value=4000, step=500)

        x = Dense(layer_nodes, activation=layer_activation)(x)
        x = Dropout(droprate)(x)

    # final Dense layers
    x = Dense(500, activation='relu')(x)
    x = Dropout(droprate)(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(droprate)(x)

    x = Dense(final_nodes, activation='relu')(x) 

    cnn_model = Model(inputs=vgg_model.input, outputs=x)

    # combine models
    combined_model = tf.keras.layers.concatenate([nn_model.output, cnn_model.output],axis=1)
    combined_model = Dense(final_nodes, activation='relu')(combined_model)

    # final output layer
    combined_model = Dense(1, activation='sigmoid')(combined_model)

    final_model = Model(inputs=[nn_model.input, cnn_model.input], outputs=combined_model)

    # sample learning rate and epsilon
    lr = hp.Float("learning_rate", min_value=1e-4, max_value=1e-1, sampling="log")
    ep = hp.Float('epsilon', min_value=1e-5, max_value=1e-3, sampling='log')

    opt = Adam(learning_rate=lr, epsilon=ep)

    # compile model
    final_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return final_model

In [3]:
def cat_kt_model_build(hp):
    """
    Creates Keras-Tuner model for parameter searching/tuning for a multiclass model (low, medium, high output layer). 
    This function has a trimmed set of search parameters.

    Paramters: 
        None

    return: 
        Keras model - combined VGG16 and neural network model with selected search parameters 
    """

    # build basic NN first - same general architecture as kt_model_build function except different output layer
    nn_model = Sequential()

    activation_list = ['relu', 'gelu', 'selu']
    # input nodes have to be same # as total features, i.e., 4
    nn_input_nodes = 4

    input_activation = hp.Choice('input_activation', activation_list)
    nn_model.add(Dense(nn_input_nodes, input_dim=nn_input_nodes, activation=input_activation))

    # generate hidden layers with nodes between 3-36 
    for i in range(hp.Int('num_layers', 3, 5)):

        layer_activation = hp.Choice(f'activation_{i}', activation_list)
        nn_model.add(Dense(units=hp.Int(f'nodes_{i}', min_value=3, max_value=36, step=3),
                        activation=layer_activation
        ))
    
    # final NN layer
    nn_model.add(Dense(3, activation='relu'))


    # build CNN
    vgg_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', pooling='avg')

    for layer in vgg_model.layers:
        layer.trainable = False

    x = vgg_model.output
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)

    # set dropout rate for dropout layers
    droprate = hp.Float('droprate', min_value=0.15, max_value=0.35, step=0.05)
    x = Dropout(droprate)(x)

    # generate dense layers with nodes between 1000-4000, dropout layers after each
    for i in range(hp.Int('cnn_set1_num_layers', 3, 6)):
        layer_activation = hp.Choice(f'cnn_set1_activation_{i}', activation_list)
        layer_nodes = hp.Int(f'cnn_set1_nodes_{i}', min_value=1000, max_value=4000, step=500)

        x = Dense(layer_nodes, activation=layer_activation)(x)
        x = Dropout(droprate)(x)

    # final dense layers
    x = Dense(500, activation='relu')(x)
    x = Dropout(droprate)(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(droprate)(x)

    # final CNN layer
    x = Dense(3, activation='relu')(x)

    cnn_model = Model(inputs=vgg_model.input, outputs=x)

    # combine models
    combined_model = tf.keras.layers.concatenate([nn_model.output, cnn_model.output],axis=1)
    combined_model = Dense(3, activation='relu')(combined_model)

    # final output layer - 3 nodes, softmax activation
    combined_model = Dense(3, activation='softmax')(combined_model)

    final_model = Model(inputs=[nn_model.input, cnn_model.input], outputs=combined_model)

    # sample learning rate and epsilon
    lr = hp.Float("learning_rate", min_value=1e-4, max_value=1e-1, sampling="log")
    ep = hp.Float('epsilon', min_value=1e-7, max_value=1e-4, sampling='log')

    opt = Adam(learning_rate=lr, epsilon=ep)

    # compile model - set loss to 'sparse_categorical_crossentropy' for multiclass
    final_model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return final_model

In [4]:
def extra_trimmed_kt_model_build(hp):
    """
    Creates Keras-Tuner model for parameter searching/tuning. This function has the most limited set of search parameters.

    Paramters: 
        None

    return: 
        Keras model - combined VGG16 and neural network model with selected search parameters 
    """

    # build basic NN first - same general architecture as kt_model_build function
    nn_model = Sequential()

    activation_list = ['relu', 'gelu', 'selu']

    # input nodes have to be same # as total features, i.e., 4
    nn_input_nodes = 4

    # activation function to use throughout model
    input_activation = hp.Choice('input_activation', activation_list)

    nn_model.add(Dense(nn_input_nodes, input_dim=nn_input_nodes, activation=input_activation))

    # add 3 hidden layers with nodes between 3-36
    nn_model.add(Dense(units=hp.Int(f'nodes_1', min_value=3, max_value=36, step=3),
                    activation=input_activation
    ))
    nn_model.add(Dense(units=hp.Int(f'nodes_2', min_value=3, max_value=36, step=3),
                    activation=input_activation
    ))
    nn_model.add(Dense(units=hp.Int(f'nodes_3', min_value=3, max_value=36, step=3),
                    activation=input_activation
    ))
    
    # final NN layer
    final_nodes = 4
    nn_model.add(Dense(final_nodes, activation='relu'))


    # build CNN
    vgg_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', pooling='avg')

    for layer in vgg_model.layers:
        layer.trainable = False

    x = vgg_model.output
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)

    # set dropout rate between 0.15-0.35
    droprate = hp.Float('droprate', min_value=0.15, max_value=0.35, step=0.05)
    x = Dropout(droprate)(x)

    # create 3 layers with nodes between 1000-4000 and dropout layers between them
    layer1_nodes = hp.Int(f'cnn_nodes_1', min_value=1000, max_value=4000, step=500)

    x = Dense(layer1_nodes, activation=input_activation)(x)
    x = Dropout(droprate)(x)

    layer2_nodes = hp.Int(f'cnn_nodes_2', min_value=1000, max_value=4000, step=500)
    x = Dense(layer2_nodes, activation=input_activation)(x)
    x = Dropout(droprate)(x)

    layer3_nodes = hp.Int(f'cnn_nodes_3', min_value=1000, max_value=4000, step=500)
    x = Dense(layer3_nodes, activation=input_activation)(x)
    x = Dropout(droprate)(x)

    # final CNN dense/dropout layers
    x = Dense(500, activation=input_activation)(x)
    x = Dropout(droprate)(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(droprate)(x)

    # final CNN layer
    x = Dense(final_nodes, activation='relu')(x)

    cnn_model = Model(inputs=vgg_model.input, outputs=x)

    # combine models
    combined_model = tf.keras.layers.concatenate([nn_model.output, cnn_model.output],axis=1)
    combined_model = Dense(final_nodes, activation='relu')(combined_model)

    # final output layer
    combined_model = Dense(1, activation='sigmoid')(combined_model)

    final_model = Model(inputs=[nn_model.input, cnn_model.input], outputs=combined_model)

    # sample learning rate and epsilon values
    lr = hp.Float("learning_rate", min_value=1e-4, max_value=1e-1, sampling="log")
    ep = hp.Float('epsilon', min_value=1e-5, max_value=1e-3, sampling='log')

    opt = Adam(learning_rate=lr, epsilon=ep)

    # compile model
    final_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return final_model